# Example       

In [3]:
import cvxpy as cp
import numpy as np

m = 10
n = 5

np.random.seed(1)
A = np.random.randn(m,n)
b = np.random.randn(m)

x = cp.Variable(n)
cost = cp.sum_squares(A*x - b)
objective = cp.Minimize(cost)
constraints = [0 <= x, x <=1]
prob= cp.Problem(objective, constraints)

result = prob.solve()

print("optimal parameter:\n", x.value)
print("Lagrange parameter\n", constraints[0].dual_value)
print("status:"+ prob.status)

optimal parameter:
 [-4.95922264e-21  6.07571976e-21  1.34643668e-01  1.24976681e-01
 -4.57130806e-21]
Lagrange parameter
 [2.00105768 0.75536127 0.         0.         1.17911779]
status:optimal


In [11]:
A = np.array([[1,1,1,1,1],
             [2,2,2,2,2]])
cost

Expression(CONVEX, NONNEGATIVE, ())

In [2]:
import cvxopt
from cvxopt import matrix
import numpy as np

P = matrix(np.diag([1.0, -1.0]))
q = matrix(np.array([3.0, 4.0]))
A = matrix(np.array([[-1.0, 0.0],[2.0,3.0]]).astype(np.float))
b = matrix(np.array([1.0,1.0]))
G = matrix(np.array([[-2.0, 0.0]]).astype(np.float))
h = matrix(np.array([3.0]))

sol = cvxopt.solvers.qp(P,q, A=A, b=b,G=G, h=h)
print(sol)
print(sol['x'])
print(sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0:  1.0000e+00  8.8818e-16  1e+00  4e-16  8e-01
 1:  1.0000e+00  9.9000e-01  1e-02  8e-17  8e-03
 2:  1.0000e+00  9.9990e-01  1e-04  1e-16  8e-05
 3:  1.0000e+00  1.0000e+00  1e-06  1e-16  8e-07
 4:  1.0000e+00  1.0000e+00  1e-08  7e-17  8e-09
Optimal solution found.
{'x': <2x1 matrix, tc='d'>, 'y': <2x1 matrix, tc='d'>, 's': <1x1 matrix, tc='d'>, 'z': <1x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.0000000000000037e-08, 'relative gap': 1.0000000100000038e-08, 'primal objective': 1.0, 'dual objective': 0.99999999, 'primal infeasibility': 7.401486830834377e-17, 'dual infeasibility': 8.000000064507973e-09, 'primal slack': 1.0000000000000002, 'dual slack': 1.0000000000000035e-08, 'iterations': 4}
[-1.00e+00]
[ 1.00e+00]

1.0


In [3]:
import cvxpy as cp

# Create 2 scalar optimization value
x = cp.Variable()
y = cp.Variable()

# Create 2 constrains
constrains = [ (x + y) == 1,
               (x - y) >= 1]

# Form objective(目的関数・コスト関数)
cost = (x- y)**2
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constrains)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)


status: optimal
optimal value 1.0
optimal value 1.0 1.570086213240983e-22


In [4]:
import cvxpy as vp

# Creaate 2 scalar optimization value
x1 = cp.Variable()
x2 = cp.Variable()

# Create 2 constraints
constraints = [(x1 + x2) == 1,
               (x1 - x2) >= 1]

# Form objective
cost = -x1+x2
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)

status: unbounded
optimal value -inf
optimal value 1.0 1.570086213240983e-22


In [5]:
import cvxpy as cp

# Create Variables
x = cp.Variable()
y = cp.Variable()

# Create constraints
constraints = [(x + y) >= 1]

# Cost function
cost = x**2 + y**2
# Form objective
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj,constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)


status: optimal
optimal value 0.5000000000000002
optimal value 0.5000000000000001 0.5000000000000001


In [6]:

import cvxpy as cp

# Create Variables
x1 = cp.Variable()
x2 = cp.Variable()

# Create constraints
constraints = [x1 <= 5,
               -x1<= 5,
               x2 <= 5,
               -x2<= 5]

# Cost
cost = (x1 - 6)**2 + 2+x2**2
# objective
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x1.value, x2.value)


status: optimal
optimal value 3.000000000000001
optimal value 5.0 0.0


# Model predoctive control with cvxpy

In [7]:
import time
from cvxpy import *
import numpy as np
import matplotlib.pyplot as plt
print("Simulation start")

np.random.seed(1)
n = 4   #state size
m = 2   #input size
T = 50  #number of horizon

#simulation parameter
alpha = 0.2
beta = 5.0

# Model Parameter
A = np.eye(n) + alpha*np.random.randn(n,n)
B = np.random.randn(n,m)
x_0 = beta*np.random.randn(n,1)

x = cp.Variable(shape=(n, T+1))
u = cp.Variable(shape=(m, T))

states = []
for t in range(T):
    cost = sum_squares(x[:,t+1]) + sum_squares(u[:,t])
    constr = [x[:,t+1] == A*x[:,t] + B*u[:,t],
              norm(u[:,t], 'inf') <= 1]
    states.append( Problem(Minimize(cost), constr) )
# sums problem objectives and concatenates constraints.
prob = sum(states)
#prob.constraints = [x[:,0] == x_0]
#prob.constraints += [x[:,T+1] == 0,
#                     x[:,0] == x_0]

start = time.time()
result=prob.solve(verbose=True)
elapsed_time = time.time() - start
print ("calc time:{} [s]".format(elapsed_time))

if result == float("inf"):
    print("Cannot optimize")
    import sys
    sys.exit()

f = plt.figure()
ax = f.add_subplot(211)
u1=np.array(u[0,:].value[0,:])[0].tolist()
u2=np.array(u[1,:].value[0,:])[0].tolist()
plt.plot(u1,'-r',label="u1")
plt.plot(u2,'-b',label="u2")
plt.ylabel(r"$u_t$", fontsize=16)
plt.yticks(np.linspace(-1.0, 1.0, 3))
plt.legend()
plt.grid(True)

plt.subplot(2,1,2)
x1=np.array(x[0,:].value[0,:])[0].tolist()
x2=np.array(x[1,:].value[0,:])[0].tolist()
x3=np.array(x[2,:].value[0,:])[0].tolist()
x4=np.array(x[3,:].value[0,:])[0].tolist()
plt.plot(range(T+1), x1,'-r',label="x1")
plt.plot(range(T+1), x2,'-b',label="x2")
plt.plot(range(T+1), x3,'-g',label="x3")
plt.plot(range(T+1), x4,'-k',label="x4")
plt.yticks([-25, 0, 25])
plt.ylim([-25, 25])
plt.ylabel(r"$x_t$", fontsize=16)
plt.xlabel(r"$t$", fontsize=16)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Simulation start
-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 654, constraints m = 750
          nnz(P) + nnz(A) = 2750
settings: linear system solver = qdldl,
          eps_abs = 1.0e-04, eps_rel = 1.0e-04,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

objective    pri res    dua res    rho        time
   1   0.0000e+00   0.00e+00   0.00e+00   1.00e-01   5.90e-04s
  25   0.0000e+00   0.00e+00   0.00e+00   1.00e-01   1.31e-03s

status:               solved
solution polish:     

IndexError: too many indices for array

In [148]:
import time
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

n_state = 4   # 状態の数
m_state = 1   # 制御入力の数
T = 100  # 何ステップ先まで予測するかを決める

#simulation parameter
delta_t = 0.01

M = 1.0  # [kg]
m = 0.3  # [kg]
g = 9.8  # [m/s^2]
l = 0.6  # [m]

# Model Parameter
A = np.array([
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, m * g / M, 0.0],
    [0.0, 0.0, 0.0, 1.0],
    [0.0, 0.0, g * (M + m) / (l * M), 0.0]
    ])
A = np.eye(n_state) + delta_t * A

B = np.array([
    [0.0],
    [1.0 / M],
    [0.0],
    [1.0 / (l * M)]
    ])
B = delta_t * B

# 倒立振子の初期状態
# 今回はすべてが0に収束するよう目指す
""" 
# もとの行列だと次元の違いでエラーになる
x0 = np.array([
    [-0.02],
    [0.0],
    [0.02],
    [0.0]
    ])
"""
x0 = np.array([-0.02, 0.0, 0.02,0.0])
#x0.shape

In [ ]:
x = cp.Variable((n_state, T+1))
u = cp.Variable((m_state, T))

cost_arr_Q = np.array([
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.1]
    ])

cost_arr_R = np.array([0.1, 0.1, 0.1, 0.1])

#states = []
for pred_step in range(T):
    # コスト関数の値が小さくなるような配列uを求める
    # cost += cp.sum_squares(cost_arr@x[:,pred_step+1]) + sum_squares(0.1*u[:,pred_step])
    cost += cp.sum_squares(cost_arr@x[:,pred_step+1]) + sum_squares(cost_arr_R@u[:,pred_step])

    # 制約式（線形方程式と制御入力の限界値）を与える
    constraints += [x[:,t+1] == A@x[:,t] + B@u[:,t], # 状態方程式の制約
                    cp.norm(u[:,t], 'inf') <= 20.0]  # 入力制限の制約
     
    # Push formed problems
    #states.append(prob)
    #states.append( Problem(Minimize(cost), constraints) )

# sums problem objectives and concatenates constraints.
constraints += [x[:,T] == 0, x[:,0] == x0]

# 最適化の目的関数
obj = cp.Minimize(cost)
# Form problem
prob = cp.Problem(obj, constraints)

# sums problem objectives and concatenates constraints.
#prob = cp.sum(states)
# 制約をさらに2つ追加する
# 最後の状態(Tステップ後の状態)はすべてが0、すなわち理想の状態とすること
# そして、現在の状態x0は事実としてあるので、制約となる
#prob.constraints += [x[:,T] == 0, x[:,0:1] == x0]
#prob.constraints += [x[:,T] == 0]
#prob.constraints += [x[:,0:1] == x0]

start = time.time()
result=prob.solve(verbose=False)
elapsed_time = time.time() - start
print("calc time:{0} [sec]".format(elapsed_time))

# 発散した場合は制御不能として終了
if result == float("inf"):
    print("Cannot optimize")
    import sys
    sys.exit()
    
# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", u[:,0].value, u[:,T-1].value)

In [111]:
x[:,0:1].shape

(4, 1)

In [135]:
# Generate data for control problem.
import numpy as np
np.random.seed(1)
n = 8
m = 2
T = 50
alpha = 0.2
beta = 5
A = np.eye(n) + alpha*np.random.randn(n,n)
B = np.random.randn(n,m)
x_0 = beta*np.random.randn(n)

In [137]:
# Form and solve control problem.
import cvxpy as cp


x = cp.Variable((n, T+1))
u = cp.Variable((m, T))

cost = 0
constr = []
for t in range(T):
    cost += cp.sum_squares(x[:,t+1]) + cp.sum_squares(u[:,t])
    constr += [x[:,t+1] == A@x[:,t] + B@u[:,t],
               cp.norm(u[:,t], 'inf') <= 1]
# sums problem objectives and concatenates constraints.
constr += [x[:,T] == 0, x[:,0] == x_0]
problem = cp.Problem(cp.Minimize(cost), constr)
problem.solve(solver=cp.ECOS)

64470.57722638684

In [ ]:
"""
# Plot results.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

f = plt.figure()

# Plot (u_t)_1.
ax = f.add_subplot(411)
plt.plot(u[0,:].value)
plt.ylabel(r"$(u_t)_1$", fontsize=16)
plt.yticks(np.linspace(-1.0, 1.0, 3))
plt.xticks([])

# Plot (u_t)_2.
plt.subplot(4,1,2)
plt.plot(u[1,:].value)
plt.ylabel(r"$(u_t)_2$", fontsize=16)
plt.yticks(np.linspace(-1, 1, 3))
plt.xticks([])

# Plot (x_t)_1.
plt.subplot(4,1,3)
x1 = x[0,:].value
plt.plot(x1)
plt.ylabel(r"$(x_t)_1$", fontsize=16)
plt.yticks([-10, 0, 10])
plt.ylim([-10, 10])
plt.xticks([])

# Plot (x_t)_2.
plt.subplot(4,1,4)
x2 = x[1,:].value
plt.plot(range(51), x2)
plt.yticks([-25, 0, 25])
plt.ylim([-25, 25])
plt.ylabel(r"$(x_t)_2$", fontsize=16)
plt.xlabel(r"$t$", fontsize=16)
plt.tight_layout()
plt.show()
"""

In [125]:
print( x[:,0:1] == x0)

var104627[0:4, 0] == [[-0.02]
 [ 0.  ]
 [ 0.02]
 [ 0.  ]]


In [138]:
x_0.shape

(8,)

In [139]:
x[:,0].shape

(8,)

In [140]:
x_0

array([-1.11164071, -1.00379034,  0.93280695,  2.05025824,  0.9914986 ,
        0.59504323, -3.35331143,  1.88781893])

In [59]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import cvxpy
from matplotrecorder import matplotrecorder
%inline matplotllib

l_bar = 2.0  # length of bar
M = 1.0  # [kg]
m = 0.3  # [kg]
g = 9.8  # [m/s^2]

Q = np.diag([0.0, 1.0, 1.0, 0.0])
R = np.diag([0.01])
nx = 4
nu = 1   
T = 30  
delta_t = 0.1

animation = False


def main():
    x0 = np.array([
        [0.0],
        [0.0],
        [0.3],
        [0.0]
    ])

    x = np.copy(x0)

    for i in range(50):

        ox, dx, otheta, dtheta, ou = mpc_control(x)
        u = ou[0]
        x = simulation(x, u)
        print(i)
        print(x)
        print(u)

        plt.clf()
        px = float(x[0])
        theta = float(x[2])
        show_cart(px, theta)
        plt.xlim([-5.0, 2.0])
        plt.pause(0.001)

        if animation:
            matplotrecorder.save_frame()

    if animation:
        matplotrecorder.save_movie("animation.gif", 0.1)


def simulation(x, u):

    A, B = get_model_matrix()

    x = np.dot(A, x) + np.dot(B, u)

    return x


def mpc_control(x0):

    x = cvxpy.Variable(nx, T + 1)
    u = cvxpy.Variable(nu, T)

    A, B = get_model_matrix()

    cost = 0.0
    constr = []
    for t in range(T):
        cost += cvxpy.quad_form(x[:, t + 1], Q)
        cost += cvxpy.quad_form(u[:, t], R)
        constr += [x[:, t + 1] == A * x[:, t] + B * u[:, t]]
    constr += [x[:, 0] == x0]
    prob = cvxpy.Problem(cvxpy.Minimize(cost), constr)

    start = time.time()
    prob.solve(verbose=False)
    elapsed_time = time.time() - start
    print("calc time:{0} [sec]".format(elapsed_time))

    if prob.status == cvxpy.OPTIMAL:
        ox = get_nparray_from_matrix(x.value[0, :])
        dx = get_nparray_from_matrix(x.value[1, :])
        theta = get_nparray_from_matrix(x.value[2, :])
        dtheta = get_nparray_from_matrix(x.value[3, :])

        ou = get_nparray_from_matrix(u.value[0, :])

    return ox, dx, theta, dtheta, ou


def get_nparray_from_matrix(x):
    u"""
    get build-in list from matrix
    """
    return np.array(x).flatten()


def get_model_matrix():

    # Model Parameter
    A = np.array([
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, m * g / M, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, g * (M + m) / (l_bar * M), 0.0]
    ])
    A = np.eye(nx) + delta_t * A

    B = np.array([
        [0.0],
        [1.0 / M],
        [0.0],
        [1.0 / (l_bar * M)]
    ])
    B = delta_t * B

    return A, B


def flatten(a):
    return np.array(a).flatten()


def show_cart(xt, theta):
    cart_w = 1.0
    cart_h = 0.5
    radius = 0.1

    cx = np.matrix([-cart_w / 2.0, cart_w / 2.0, cart_w /
                    2.0, -cart_w / 2.0, -cart_w / 2.0])
    cy = np.matrix([0.0, 0.0, cart_h, cart_h, 0.0])
    cy += radius * 2.0

    cx = cx + xt

    bx = np.matrix([0.0, l_bar * math.sin(-theta)])
    bx += xt
    by = np.matrix([cart_h, l_bar * math.cos(-theta) + cart_h])
    by += radius * 2.0

    angles = np.arange(0.0, math.pi * 2.0, math.radians(3.0))
    ox = [radius * math.cos(a) for a in angles]
    oy = [radius * math.sin(a) for a in angles]

    rwx = np.copy(ox) + cart_w / 4.0 + xt
    rwy = np.copy(oy) + radius
    lwx = np.copy(ox) - cart_w / 4.0 + xt
    lwy = np.copy(oy) + radius

    wx = np.copy(ox) + float(bx[0, -1])
    wy = np.copy(oy) + float(by[0, -1])

    plt.plot(flatten(cx), flatten(cy), "-b")
    plt.plot(flatten(bx), flatten(by), "-k")
    plt.plot(flatten(rwx), flatten(rwy), "-k")
    plt.plot(flatten(lwx), flatten(lwy), "-k")
    plt.plot(flatten(wx), flatten(wy), "-k")
    plt.title("x:" + str(round(xt, 2)) + ",theta:" +
              str(round(math.degrees(theta), 2)))

    plt.axis("equal")


ModuleNotFoundError: No module named 'matplotrecorder'

In [67]:
l_bar = 2.0  # length of bar
M = 1.0  # [kg]
m = 0.3  # [kg]
g = 9.8  # [m/s^2]

Q = np.diag([0.0, 1.0, 1.0, 0.0])
R = np.diag([0.01])
nx = 4
nu = 1   
T = 30  
delta_t = 0.1

animation = False

def get_model_matrix():

    # Model Parameter
    A = np.array([
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, m * g / M, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, g * (M + m) / (l_bar * M), 0.0]
    ])
    A = np.eye(nx) + delta_t * A

    B = np.array([
        [0.0],
        [1.0 / M],
        [0.0],
        [1.0 / (l_bar * M)]
    ])
    B = delta_t * B

    return A, B


def mpc_control(x0):

    x = cvxpy.Variable((nx, T + 1))
    u = cvxpy.Variable((nu, T))

    A, B = get_model_matrix()

    cost = 0.0
    constr = []
    for t in range(T):
        cost += cvxpy.quad_form(x[:, t + 1], Q)
        cost += cvxpy.quad_form(u[:, t], R)
        constr += [x[:, t + 1] == A * x[:, t] + B * u[:, t]]
    constr += [x[:, 0] == x0]
    prob = cvxpy.Problem(cvxpy.Minimize(cost), constr)

    start = time.time()
    prob.solve(verbose=False)
    elapsed_time = time.time() - start
    print("calc time:{0} [sec]".format(elapsed_time))

    if prob.status == cvxpy.OPTIMAL:
        ox = get_nparray_from_matrix(x.value[0, :])
        dx = get_nparray_from_matrix(x.value[1, :])
        theta = get_nparray_from_matrix(x.value[2, :])
        dtheta = get_nparray_from_matrix(x.value[3, :])

        ou = get_nparray_from_matrix(u.value[0, :])

    return ox, dx, theta, dtheta, ou


In [68]:
x0 = np.array([
        [0.0],
        [0.0],
        [0.3],
        [0.0]
    ])

ox, dx, theta, dtheta, ou = mpc_control(x0)

ValueError: Cannot broadcast dimensions  (4,) (4, 1)